In [1]:
# import pandas as pd
# from scipy import stats
# import numpy as np

# # 1. Load the Two Generated CSV Files
# # 1. Load the Data
# file_zs = 'qwen2_ft_zs_full_metrics.csv'
# file_fs = 'qwen2_ft_fs_full_metrics.csv'

# df_zs = pd.read_csv(file_zs)
# df_fs = pd.read_csv(file_fs)

# # 2. Ensure Images align (Sort by Image ID just in case)
# df_zs = df_zs.sort_values('image_id').reset_index(drop=True)
# df_fs = df_fs.sort_values('image_id').reset_index(drop=True)

# # 3. Define Metrics List
# metrics = [
#     'BLEU-1', 'BLEU-2', 'BLEU-3', 'BLEU-4',
#     'ROUGE-1', 'ROUGE-2', 'ROUGE-L',
#     'METEOR', 'CIDEr',
#     'BERTScore', 'CLIPScore', 'RefCLIPScore',
#     'Distinct-1', 'Distinct-2'
# ]

# # 4. Perform Wilcoxon Test
# print(f"{'Metric':<15} | {'P-Value':<12} | {'Verdict'}")
# print("-" * 50)

# for metric in metrics:
#     # Get the two lists of scores
#     scores_zs = df_zs[metric]
#     scores_fs = df_fs[metric]

#     # Perform Wilcoxon Signed-Rank Test
#     # 'two-sided' checks for ANY significant difference
#     stat, p_value = stats.wilcoxon(scores_zs, scores_fs)

#     # Determine Significance
#     if p_value < 0.05:
#         # Check Direction (Did it improve or degrade?)
#         mean_diff = scores_fs.mean() - scores_zs.mean()
#         direction = "Improved (+)" if mean_diff > 0 else "Degraded (-)"
#         verdict = f"✅ Sig ({direction})"
#     else:
#         verdict = "❌ Not Sig"

#     print(f"{metric:<15} | {p_value:.2e}     | {verdict}")

In [2]:
# import pandas as pd
# import numpy as np
# from scipy import stats

# # 1. Load the Data
# file_zs = 'qwen2_ft_zs_full_metrics.csv'
# file_fs = 'qwen2_ft_fs_full_metrics.csv'

# df_zs = pd.read_csv(file_zs)
# df_fs = pd.read_csv(file_fs)

# # Ensure alignment
# df_zs = df_zs.sort_values('image_id').reset_index(drop=True)
# df_fs = df_fs.sort_values('image_id').reset_index(drop=True)

# metrics = [
#     'BLEU-1', 'BLEU-2', 'BLEU-3', 'BLEU-4',
#     'ROUGE-1', 'ROUGE-2', 'ROUGE-L',
#     'METEOR', 'CIDEr',
#     'BERTScore', 'CLIPScore', 'RefCLIPScore',
#     'Distinct-1', 'Distinct-2'
# ]

# # 2. Bootstrap Function for 95% CI
# def get_bootstrap_ci(data, n_bootstrap=1000, ci=0.95):
#     """Calculates the 95% Confidence Interval for the mean."""
#     means = []
#     n = len(data)
#     for _ in range(n_bootstrap):
#         # Sample with replacement
#         sample = np.random.choice(data, size=n, replace=True)
#         means.append(np.mean(sample))

#     lower = np.percentile(means, (1 - ci) / 2 * 100)
#     upper = np.percentile(means, (1 + ci) / 2 * 100)
#     return lower, upper

# # 3. Main Loop
# p_value_data = []
# ci_data = []

# for metric in metrics:
#     scores_zs = df_zs[metric].values
#     scores_fs = df_fs[metric].values

#     # --- A. P-Values (Wilcoxon) ---
#     stat, p_val = stats.wilcoxon(scores_zs, scores_fs)

#     # Verdict Logic
#     if p_val < 0.05:
#         verdict = "Significant"
#         direction = "Improved (+)" if np.mean(scores_fs) > np.mean(scores_zs) else "Degraded (-)"
#     else:
#         verdict = "Not Significant"
#         direction = "-"

#     p_value_data.append({
#         'Metric': metric,
#         'P_Value': p_val,
#         'Significance': verdict,
#         'Direction': direction
#     })

#     # --- B. Confidence Intervals (Bootstrapping) ---
#     zs_mean = np.mean(scores_zs)
#     fs_mean = np.mean(scores_fs)

#     zs_ci_low, zs_ci_high = get_bootstrap_ci(scores_zs)
#     fs_ci_low, fs_ci_high = get_bootstrap_ci(scores_fs)

#     ci_data.append({
#         'Metric': metric,
#         'ZS_Mean': zs_mean,
#         'ZS_CI_Lower': zs_ci_low,
#         'ZS_CI_Upper': zs_ci_high,
#         'FS_Mean': fs_mean,
#         'FS_CI_Lower': fs_ci_low,
#         'FS_CI_Upper': fs_ci_high
#     })

# # 4. Save to 2 Separate CSV Files
# df_p_values = pd.DataFrame(p_value_data)
# df_cis = pd.DataFrame(ci_data)

# df_p_values.to_csv('P_Values_qwen2_ft(zs)_vs_ft(fs).csv', index=False)
# df_cis.to_csv('CIs_qwen2_ft(zs)_vs_ft(fs).csv', index=False)

# print("File 1: Wilcoxon_P_Values.csv")
# print(df_p_values)
# print("\n" + "="*50 + "\n")
# print("File 2: Confidence_Intervals.csv")
# print(df_cis)

# **NEW-FORMAT**

# **CIs Scores**

In [ ]:
import pandas as pd
import numpy as np

# ==========================================
# INPUT: Set your file path here
# ==========================================
input_file = "LLama_FT_zs_full_metrics.csv"
output_file = "Llama_Ft_zs_CIs_Individual.csv"
# ==========================================

metrics = [
    'BLEU-1', 'BLEU-2', 'BLEU-3', 'BLEU-4',
    'ROUGE-1', 'ROUGE-2', 'ROUGE-L',
    'METEOR', 'CIDEr',
    'BERTScore', 'CLIPScore', 'RefCLIPScore',
    'Distinct-1', 'Distinct-2'
]

def get_bootstrap_ci(data, n_bootstrap=10000, ci=0.95):
    """
    Calculates Mean and 95% CI using Bootstrapping.
    n_bootstrap=10000 is standard for high-impact journals.
    """
    data = np.array(data)
    # 1. True Mean (matches your Master Table)
    mean_val = np.mean(data)

    # 2. Bootstrap
    means = []
    n = len(data)
    for _ in range(n_bootstrap):
        # Sample with replacement
        sample = np.random.choice(data, size=n, replace=True)
        means.append(np.mean(sample))

    lower = np.percentile(means, (1 - ci) / 2 * 100)
    upper = np.percentile(means, (1 + ci) / 2 * 100)

    return mean_val, lower, upper

# Load Data
df = pd.read_csv(input_file)
print(f"Loaded {len(df)} rows from {input_file}")

results = []
print(f"{'Metric':<15} | {'Mean':<10} | {'95% CI'}")
print("-" * 45)

for metric in metrics:
    if metric in df.columns:
        scores = df[metric].dropna().values
        mean_val, low, high = get_bootstrap_ci(scores)

        # Formatting
        ci_str = f"[{low:.3f}, {high:.3f}]"
        print(f"{metric:<15} | {mean_val:<10.3f} | {ci_str}")

        results.append({
            'Metric': metric,
            'Mean': mean_val,
            'CI_Lower': low,
            'CI_Upper': high,
            'Formatted_CI': ci_str
        })

# Save
pd.DataFrame(results).to_csv(output_file, index=False)
print(f"\n[Done] Saved to {output_file}")

Loaded 100 rows from LLama_FT_zs_full_metrics.csv
Metric          | Mean       | 95% CI
---------------------------------------------
BLEU-1          | 0.855      | [0.831, 0.879]
BLEU-2          | 0.685      | [0.652, 0.718]
BLEU-3          | 0.550      | [0.511, 0.588]
BLEU-4          | 0.427      | [0.384, 0.470]
ROUGE-1         | 0.667      | [0.643, 0.692]
ROUGE-2         | 0.451      | [0.417, 0.487]
ROUGE-L         | 0.636      | [0.608, 0.664]
METEOR          | 0.637      | [0.607, 0.667]
CIDEr           | 1.147      | [1.004, 1.299]
BERTScore       | 0.706      | [0.685, 0.727]
CLIPScore       | 33.728     | [33.158, 34.294]
RefCLIPScore    | 87.902     | [86.511, 89.206]
Distinct-1      | 0.379      | [0.366, 0.392]
Distinct-2      | 0.621      | [0.600, 0.642]

[Done] Saved to Llama_Ft_zs_CIs_Individual.csv


# **P-TEST SCORES**

In [ ]:
import pandas as pd
from scipy import stats

# ==========================================
# INPUTS: Set your comparison files here
# ==========================================
file_baseline = "qwen3_base_zs_full_metrics.csv"  # Baseline (e.g., Zero-Shot)
file_target   = "qwen3_base_fs_full_metrics.csv"  # Target (e.g., Few-Shot)
output_file   = "P_Values_qwen3_base(zs)_vs_base(fs).csv"
# ==========================================

metrics = [
    'BLEU-1', 'BLEU-2', 'BLEU-3', 'BLEU-4',
    'ROUGE-1', 'ROUGE-2', 'ROUGE-L',
    'METEOR', 'CIDEr',
    'BERTScore', 'CLIPScore', 'RefCLIPScore',
    'Distinct-1', 'Distinct-2'
]

# Load Data
df_base = pd.read_csv(file_baseline)
df_tgt = pd.read_csv(file_target)

# Merge on image_id to ensure strict pairing
# (Comparing Image 1 from Model A vs Image 1 from Model B)
merged = pd.merge(df_base, df_tgt, on='image_id', suffixes=('_B', '_T'))
print(f"Aligned {len(merged)} images for comparison.")

results = []
print(f"{'Metric':<15} | {'P-Value':<15} | {'Verdict'}")
print("-" * 50)

for metric in metrics:
    col_b = f"{metric}_B"
    col_t = f"{metric}_T"

    if col_b in merged.columns and col_t in merged.columns:
        scores_base = merged[col_b].values
        scores_tgt = merged[col_t].values

        # 1. Wilcoxon Signed-Rank Test
        try:
            stat, p_val = stats.wilcoxon(scores_base, scores_tgt)
        except ValueError:
            p_val = 1.0 # Happens if all scores are exactly identical

        # 2. Determine Direction
        mean_diff = scores_tgt.mean() - scores_base.mean()
        direction = "(+)" if mean_diff > 0 else "(-)"

        # 3. Format P-Value
        # Use scientific notation for very small numbers (e.g., 3.4e-10)
        if p_val < 0.001:
            p_str = f"{p_val:.2e}"
        else:
            p_str = f"{p_val:.3f}"

        label = "Sig" if p_val < 0.05 else "Not Sig"

        print(f"{metric:<15} | {p_str:<15} | {label} {direction}")

        results.append({
            'Metric': metric,
            'P_Value': p_val,
            'Formatted_P': p_str,
            'Significance': label,
            'Direction': direction
        })

# Save
pd.DataFrame(results).to_csv(output_file, index=False)
print(f"\n[Done] Saved to {output_file}")

Aligned 100 images for comparison.
Metric          | P-Value         | Verdict
--------------------------------------------------
BLEU-1          | 1.42e-04        | Sig (+)
BLEU-2          | 2.40e-10        | Sig (+)
BLEU-3          | 1.35e-09        | Sig (+)
BLEU-4          | 3.84e-08        | Sig (+)
ROUGE-1         | 1.34e-12        | Sig (+)
ROUGE-2         | 3.51e-12        | Sig (+)
ROUGE-L         | 1.07e-10        | Sig (+)
METEOR          | 1.37e-10        | Sig (+)
CIDEr           | 0.130           | Not Sig (+)
BERTScore       | 1.42e-15        | Sig (+)
CLIPScore       | 2.40e-10        | Sig (+)
RefCLIPScore    | 0.009           | Sig (+)
Distinct-1      | 3.90e-18        | Sig (-)
Distinct-2      | 3.90e-18        | Sig (-)

[Done] Saved to P_Values_qwen3_base(zs)_vs_base(fs).csv
